# 4. Spatio-temporal diversity of SOIL 

In [27]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal

In [28]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal


In [29]:
%env TMPDIR=/scratch/lfloerl/tmpdata

env: TMPDIR=/scratch/lfloerl/tmpdata


In [30]:
from qiime2 import Visualization
import qiime2 as q2
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2 import Metadata
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from qiime2.plugins.diversity.visualizers import alpha_group_significance


from qiime2.plugins import diversity as q2d

%matplotlib inline

In [31]:
md = Metadata.load('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv')
taxonomy = q2.Artifact.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/taxonomy.qza')

In [32]:
# just use the same rarefaction depth for all sample types
depth = 5000 
threads = 5

# Diversity 

In [33]:
table = q2.Artifact.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table.qza')

depth of 5000: Retained 220’000 (30.45%) features in 44 (80.00%) samples

still use because also lowering to 1000 only keeps 83 % of samples

In [34]:
Visualization.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table.qzv')

<visualization: Visualization uuid: 90e5e561-bacc-4db9-885c-7fad52b1a9dc>

In [35]:
core_div_dict = {}
core_div = q2d.actions.core_metrics(table=table,
                                    sampling_depth=depth,
                                    metadata=md, 
                                    n_jobs=threads)
# save outputs
output_dir='soil_cm{0}/'.format(depth)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for k, v in core_div.__dict__.items():
    if k != '_fields':
        v.save(output_dir + k)
        core_div_dict[k] = v

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/skbio/util/_warning.py:60: UserWarning: observed_otus is deprecated as of 0.6.0.
  warn(f"{func.__name__} is deprecated as of {ver}.")


In [36]:
Visualization.load(f'{workdir}/{output_dir}/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: a808ed42-9829-4d11-8228-6cb12982e413>

In [37]:
Visualization.load(f'{workdir}/{output_dir}/jaccard_emperor.qzv')

<visualization: Visualization uuid: cf4ce437-8a33-4cca-9ecb-eeae9cd3a30c>

## Kmerizer 

In [41]:
#!rm -r soil_kmer_cm5000

In [42]:
%%bash

qiime feature-table filter-seqs \
    --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table.qza \
    --o-filtered-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_rep_seqs.qza

qiime kmerizer core-metrics \
    --i-sequences /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_rep_seqs.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table.qza \
    --p-sampling-depth 5000 \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
    --p-n-jobs 'auto' \
    --p-max-features 5000 \
    --output-dir soil_kmer_cm5000/

Saved FeatureData[Sequence] to: /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_rep_seqs.qza
Saved FeatureTable[Frequency] to: soil_kmer_cm5000/rarefied_table.qza
Saved FeatureTable[Frequency] to: soil_kmer_cm5000/kmer_table.qza
Saved SampleData[AlphaDiversity] to: soil_kmer_cm5000/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: soil_kmer_cm5000/shannon_vector.qza
Saved DistanceMatrix to: soil_kmer_cm5000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: soil_kmer_cm5000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: soil_kmer_cm5000/jaccard_pcoa_results.qza
Saved PCoAResults to: soil_kmer_cm5000/bray_curtis_pcoa_results.qza
Saved Visualization to: soil_kmer_cm5000/scatterplot.qzv


In [43]:
Visualization.load('soil_kmer_cm5000/scatterplot.qzv')

<visualization: Visualization uuid: 98db19d3-c91d-4295-a782-0a179d6f38b0>

### Alpha Group Significance 

In [21]:
# metrics: 
alpha_metrics = ['shannon_vector', 'observed_features_vector', 'evenness_vector']

for m in alpha_metrics:
    ags, = alpha_group_significance(alpha_diversity = core_div_dict[m],
                               metadata = md)
    ags.save(output_dir + 'alpha_groupSig_{0}.qzv'.format(m))

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan

In [22]:
Visualization.load(f'{output_dir}/alpha_groupSig_evenness_vector.qzv')

<visualization: Visualization uuid: c1adcdbb-361e-4538-aa0b-653e8c578630>

In [23]:
Visualization.load(f'{output_dir}/alpha_groupSig_observed_features_vector.qzv')

<visualization: Visualization uuid: fbd0ea05-d065-44fa-8892-3d4db9981a81>

In [24]:
Visualization.load(f'{output_dir}/alpha_groupSig_shannon_vector.qzv')

<visualization: Visualization uuid: f54eba51-0196-401d-940c-76c32c2c0cdb>

### PERMANOVA

In [44]:
formula = 'Plot_ID * Year * Time_point'

beta_metrics = ['bray_curtis', 'jaccard']

# PERMANOVA with core metrics 
for m in beta_metrics:
    adonis_res, = q2d.actions.adonis(
        distance_matrix=core_div_dict[m + '_distance_matrix'],
        metadata=md,
        formula=formula,
        n_jobs=3)
    adonis_res.save(output_dir + 'adonis_{0}.qzv'.format(m))
    

# PERMANOVA with kmer core metrics 
kmer_dir = 'soil_kmer_cm5000'
    
for m in beta_metrics:
    !qiime diversity adonis \
        --i-distance-matrix "{kmer_dir}/{m}_distance_matrix.qza" \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
        --p-formula "{formula}" \
        --p-n-jobs 5 \
        --o-visualization "{kmer_dir}/adonis_{m}.qzv"

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /scratch/lfloerl/tmpdata/tmpk9ug9e_l/dm.tsv /scratch/lfloerl/tmpdata/tmpk9ug9e_l/md.tsv Plot_ID * Year * Time_point 999 3 /scratch/lfloerl/tmpdata/qiime2-temp-zwjy5z9z/adonis.tsv

R version 4.3.3 (2024-02-29) 


Lade nötiges Paket: permute
Lade nötiges Paket: lattice
This is vegan 2.6-8
Warnmeldung:
'adonis' ist veraltet.
Benutzen Sie stattdessen 'adonis2'
Siehe help("Deprecated") und help("vegan-deprecated"). 


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /scratch/lfloerl/tmpdata/tmp13px66u3/dm.tsv /scratch/lfloerl/tmpdata/tmp13px66u3/md.tsv Plot_ID * Year * Time_point 999 3 /scratch/lfloerl/tmpdata/qiime2-temp-h159q_80/adonis.tsv

R version 4.3.3 (2024-02-29) 


Lade nötiges Paket: permute
Lade nötiges Paket: lattice
This is vegan 2.6-8
Warnmeldung:
'adonis' ist veraltet.
Benutzen Sie stattdessen 'adonis2'
Siehe help("Deprecated") und help("vegan-deprecated"). 


Saved Visualization to: soil_kmer_cm5000/adonis_bray_curtis.qzv
Saved Visualization to: soil_kmer_cm5000/adonis_jaccard.qzv


In [45]:
Visualization.load(f'{output_dir}/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: b78dadc7-c384-4591-b38e-f97c4ae33dd2>

In [46]:
Visualization.load(f'{output_dir}/adonis_jaccard.qzv')

<visualization: Visualization uuid: e1871851-34bf-4074-b5b2-0e6944bf654f>

In [47]:
Visualization.load(f'{kmer_dir}/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: 236e66ac-8599-4636-af06-f956349f1dbd>

In [48]:
Visualization.load(f'{kmer_dir}/adonis_jaccard.qzv')

<visualization: Visualization uuid: b9ebf03a-f577-4f73-b9ef-d9872cb3a4af>

## Mantel 

> the geodisic distance matrix was create with q2-coordinates in a qiime2021.1 environment locally 

In [17]:
%%bash 

div_dir='soil_cm5000'
kmer_dir='soil_kmer_cm5000'
output_dir="soil_mantel"

mkdir -p $output_dir

qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $div_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $div_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $kmer_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_kmer_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $kmer_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_kmer_mantel.qzv

Saved Visualization to: soil_mantel/bray_curtis_mantel.qzv
Saved Visualization to: soil_mantel/jaccard_mantel.qzv
Saved Visualization to: soil_mantel/bray_curtis_kmer_mantel.qzv
Saved Visualization to: soil_mantel/jaccard_kmer_mantel.qzv


In [18]:
Visualization.load('soil_mantel/bray_curtis_mantel.qzv')

<visualization: Visualization uuid: 03ee830e-c718-4629-bc60-1f410e1bf080>

In [19]:
Visualization.load('soil_mantel/jaccard_mantel.qzv')

<visualization: Visualization uuid: 6da28506-ed93-44ce-a3c7-45980cce51e0>

In [20]:
Visualization.load('soil_mantel/bray_curtis_kmer_mantel.qzv')

<visualization: Visualization uuid: 55ae972b-d4f3-45ba-937c-0d6d2555fd7d>

In [21]:
Visualization.load('soil_mantel/jaccard_kmer_mantel.qzv')

<visualization: Visualization uuid: 4004ef24-a154-4da0-8b47-8beb70b135af>

<hr>

# SUBSET TO SINGLE TIMEPOINT 

# Diversity 

In [49]:
%%bash 

# subset to only harvest samples before fitting 
qiime feature-table filter-samples \
  --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
  --p-where "[Collection_ID]='Veraison_2021'" \
  --o-filtered-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_veraison21_filtered_table.qza

qiime feature-table filter-seqs \
    --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_veraison21_filtered_table.qza \
    --o-filtered-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_veraison21_filtered_rep_seqs.qza

Saved FeatureTable[Frequency] to: /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_veraison21_filtered_table.qza
Saved FeatureData[Sequence] to: /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_veraison21_filtered_rep_seqs.qza


In [50]:
table = q2.Artifact.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_veraison21_filtered_table.qza')

core_div_dict = {}
core_div = q2d.actions.core_metrics(table=table,
                                    sampling_depth=depth,
                                    metadata=md, 
                                    n_jobs=threads)
# save outputs
output_dir='soil_veraison21_cm{0}/'.format(depth)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for k, v in core_div.__dict__.items():
    if k != '_fields':
        v.save(output_dir + k)
        core_div_dict[k] = v

## Kmerizer 

In [51]:
!rm -r soil_veraison21_kmer_cm5000/

In [53]:
%%bash

qiime kmerizer core-metrics \
    --i-sequences /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_veraison21_filtered_rep_seqs.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_veraison21_filtered_table.qza \
    --p-sampling-depth 5000 \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
    --p-n-jobs 'auto' \
    --p-max-features 5000 \
    --output-dir soil_veraison21_kmer_cm5000/

Saved FeatureTable[Frequency] to: soil_veraison21_kmer_cm5000/rarefied_table.qza
Saved FeatureTable[Frequency] to: soil_veraison21_kmer_cm5000/kmer_table.qza
Saved SampleData[AlphaDiversity] to: soil_veraison21_kmer_cm5000/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: soil_veraison21_kmer_cm5000/shannon_vector.qza
Saved DistanceMatrix to: soil_veraison21_kmer_cm5000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: soil_veraison21_kmer_cm5000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: soil_veraison21_kmer_cm5000/jaccard_pcoa_results.qza
Saved PCoAResults to: soil_veraison21_kmer_cm5000/bray_curtis_pcoa_results.qza
Saved Visualization to: soil_veraison21_kmer_cm5000/scatterplot.qzv


In [54]:
Visualization.load('soil_veraison21_kmer_cm5000/scatterplot.qzv')

<visualization: Visualization uuid: e5adf44d-7af6-4c70-b897-32987c520ecd>

## Mantel 

In [55]:
%%bash 

div_dir='soil_veraison21_cm5000'
kmer_dir='soil_veraison21_kmer_cm5000'
output_dir="soil_veraison21_mantel"

mkdir -p $output_dir

qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $div_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $div_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $kmer_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_kmer_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza \
    --i-dm2 $kmer_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_kmer_mantel.qzv

Saved Visualization to: soil_veraison21_mantel/bray_curtis_mantel.qzv
Saved Visualization to: soil_veraison21_mantel/jaccard_mantel.qzv
Saved Visualization to: soil_veraison21_mantel/bray_curtis_kmer_mantel.qzv
Saved Visualization to: soil_veraison21_mantel/jaccard_kmer_mantel.qzv


In [56]:
Visualization.load('soil_veraison21_mantel/bray_curtis_mantel.qzv')

<visualization: Visualization uuid: 03c9c4e6-6a5c-4521-aaf7-2cb720c5af9b>

In [57]:
Visualization.load('soil_veraison21_mantel/jaccard_mantel.qzv')

<visualization: Visualization uuid: 7fb4b20b-dd64-4c1c-b7e1-f070df0bc35a>

In [58]:
Visualization.load('soil_veraison21_mantel/bray_curtis_kmer_mantel.qzv')

<visualization: Visualization uuid: 1ac8e6df-0e65-411b-abe2-b4b0da6b82a8>

In [59]:
Visualization.load('soil_veraison21_mantel/jaccard_kmer_mantel.qzv')

<visualization: Visualization uuid: 8b6a6a18-cb98-4ccf-af1e-7feedda397e7>

<hr>

# Random Forest


In [7]:
!ls

must_cm5000		     must_mantel
must-harvest21classify-plot  soil_cm5000
must_harvest21_cm5000	     soil_kmer_cm5000
must_harvest21_kmer_cm5000   soil_mantel
must_harvest21_mantel	     soil_veraison21_cm5000
must-harvest-classify-plot   soil_veraison21_kmer_cm5000
must_kmer_cm5000	     soil_veraison21_mantel


In [9]:
%%bash 

qiime feature-table summarize \
  --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table_filtered.qza \
  --o-visualization /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table_filtered.qzv


qiime sample-classifier classify-samples \
  --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
  --m-metadata-column Plot_ID \
  --p-random-state 666 \
  --p-n-jobs 6 \
  --output-dir ./soil-classify-plot/

Plugin error from sample-classifier:

  You have chosen to predict a metadata column that contains one or more values that match only one sample. For proper stratification of data into training and test sets, each class (value) must contain at least two samples. This is a requirement for classification problems, but stratification can be disabled for regression by setting stratify=False. Alternatively, remove all samples that bear a unique class label for your chosen metadata column. Note that disabling stratification can negatively impact predictive accuracy for small data sets.

Debug info has been saved to /scratch/lfloerl/tmpdata/qiime2-q2cli-err-5bd7snxz.log


CalledProcessError: Command 'b"\nqiime sample-classifier classify-samples \\\n  --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/soil_filtered_table.qza \\\n  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \\\n  --m-metadata-column Plot_ID \\\n  --p-random-state 666 \\\n  --p-n-jobs 6 \\\n  --p-missing-samples 'ignore' \\\n  --output-dir ./soil-classify-plot/\n"' returned non-zero exit status 1.

In [79]:
Visualization.load('must-harvest21classify-plot/accuracy_results.qzv')

<visualization: Visualization uuid: b355f394-c92a-4595-a5d0-0b548929dfa7>

In [70]:
Visualization.load('must-harvest-classify-plot/heatmap.qzv')

<visualization: Visualization uuid: eb507efa-fda1-4ef8-aca9-1d92dff40657>